In [5]:
import numpy as np
import pandas as pd
import glob
from udfs import ipca

# Ajustando dados e trazendo tudo a valor presente

In [7]:
files = glob.glob("anac.nosync/*.CSV")

In [8]:
# %%time
# for file in files:
#     df = pd.read_csv(file, sep=";", encoding='latin1')
#     df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
#     df.columns = ["ANO","MES","EMPRESA","ORIGEM","DESTINO","TARIFA","ASSENTOS"]
#     df.TARIFA = df.TARIFA.str.replace(",", ".").astype(float).copy()
#     df['REAL'] = df.apply(lambda row: ipca(row["TARIFA"], f"{row['MES']}/{row['ANO']}", "12/2022")[0], axis=1)
#     df.to_csv(f"anac.nosync/real/{file.split("/")[1]}.csv", index=False, sep=";")

# Começando com o Spark

In [9]:
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
import pyspark.sql.functions as f
from pyspark.sql.types import *


conf = SparkConf()
# conf.set("spark.eventLog.enabled", False)
# conf.set("spark.eventLog.dir", "/opt/spark/spark-events")
# conf.set("spark.history.fs.logDirectory", "/opt/spark/spark-events")
# conf.set("spark.executor.memory", "1024m")
# conf.set("spark.sql.autoBroadcastJoinThreshold", -1)
# conf.set("spark.sparkContext.setLogLevel", "ERROR")

spark = SparkSession.\
        builder.\
        appName("pyspark-anac").\
        master("spark://spark-master:7077").\
        config(conf=conf).\
        getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/11 16:38:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [10]:
spark
# sc = spark.sparkContext
# sc.addPyFile("/opt/notebooks/udfs.py")

In [11]:
schema = StructType(
    [
        StructField('ANO', IntegerType(), True), 
        StructField('MES', IntegerType(), True), 
        StructField('EMPRESA', StringType(), True), 
        StructField('ORIGEM', StringType(), True), 
        StructField('DESTINO', StringType(), True), 
        StructField('TARIFA', FloatType(), True), 
        StructField('ASSENTOS', IntegerType(), True),
        StructField('REAL', FloatType(), True),
    ]
)

In [12]:
sdf = spark.read.csv('anac.nosync/real/*.csv', header=True, sep=";", schema=schema)

In [13]:
sdf.schema

StructType([StructField('ANO', IntegerType(), True), StructField('MES', IntegerType(), True), StructField('EMPRESA', StringType(), True), StructField('ORIGEM', StringType(), True), StructField('DESTINO', StringType(), True), StructField('TARIFA', FloatType(), True), StructField('ASSENTOS', IntegerType(), True), StructField('REAL', FloatType(), True)])

In [14]:
sdf.count()

55291652

In [15]:
sdf.limit(3).show()

+----+---+-------+------+-------+------+--------+------+
| ANO|MES|EMPRESA|ORIGEM|DESTINO|TARIFA|ASSENTOS|  REAL|
+----+---+-------+------+-------+------+--------+------+
|2021| 10|    AZU|  SBAC|   SBAR| 565.9|       1|616.34|
|2021| 10|    AZU|  SBAC|   SBBR| 409.9|       1|446.44|
|2021| 10|    AZU|  SBAC|   SBBV| 748.9|       4|815.66|
+----+---+-------+------+-------+------+--------+------+



# Ajustar todos os valanac.nosync/ pela inflação de Dezembro de 2022

Nao funciona aqui por questão de processamento.


def valor_real(value, date_start):
    return ipca(value, date_start, "12/2022")[0]

vreal = f.udf(valor_real, DoubleType())    # registering as udf here


sdf = sdf.withColumn(
    'REAL',
    vreal(
        f.col("TARIFA"), 
        f.concat(f.col("MES"), f.lit("/"), f.col("ANO")),
    )
)

sdf = sdf.select("ANO", "MES", "ASSENTOS", "REAL")


# Dado final

In [16]:
data = sdf.groupby("ANO", "MES").agg(
    f.sum("ASSENTOS").alias("TOTAL"),
    f.collect_list(f.col('REAL')),
    f.collect_list(f.col('ASSENTOS')),
)

In [17]:
def get_average(values, weights):
    return float(np.average(values, weights=weights))    # function created to calculate wt. median

waverage = f.udf(get_average, DoubleType())    # registering as udf here

data = data.withColumn(
    'Weighted_average',
    waverage(
        f.col("collect_list(REAL)"), f.col("collect_list(ASSENTOS)")
    )
)

In [18]:
data.cache()

DataFrame[ANO: int, MES: int, TOTAL: bigint, collect_list(REAL): array<float>, collect_list(ASSENTOS): array<int>, Weighted_average: double]

In [19]:
data.limit(3).show()

+----+---+-------+--------------------+----------------------+-----------------+
| ANO|MES|  TOTAL|  collect_list(REAL)|collect_list(ASSENTOS)| Weighted_average|
+----+---+-------+--------------------+----------------------+-----------------+
|2022| 10|2450901|[1067.1, 660.59, ...|  [1, 2, 3, 26, 8, ...|648.7533683010134|
|2012| 10|4443441|[553.07, 1109.43,...|  [2, 1, 1, 2, 1, 7...|567.5236231638822|
|2007|  6|1366528|[459.67, 1140.66,...|  [103, 10, 56, 7, ...|681.8399205044295|
+----+---+-------+--------------------+----------------------+-----------------+



In [20]:
data.select("ANO", "MES", "TOTAL", "Weighted_average")\
    .repartition(1)\
    .write.format("com.databricks.spark.csv")\
    .option("header", "true")\
    .save("tarifa_real.csv")

AnalysisException: path file:/opt/notebooks/anac/tarifa_real.csv already exists.

In [27]:
df = data.select("ANO", "MES", "TOTAL", "Weighted_average").toPandas()

23/04/05 02:12:22 WARN DAGScheduler: Broadcasting large task binary with size 2.9 MiB


# Fechando aqui

In [4]:
from monetary_correction import MonetaryCorrection
from monetary_correction_old import MonetaryCorrection as MonetaryCorrectionOld
import pandas as pd

In [2]:
corretor = MonetaryCorrection()

In [3]:
corretorOld = MonetaryCorrectionOld()

In [5]:
dezembro = pd.read_csv("anac.nosync/201912.CSV", sep=";", encoding="latin1")

In [6]:
dezembro.columns = ['ANO', 'MES', 'EMPRESA', 'ORIGEM', 'DESTINO', 'TARIFA', 'ASSENTOS']

In [7]:
dezembro.TARIFA = dezembro.TARIFA.str.replace(",", ".").astype(float)

In [22]:
len(dezembro)

357355

In [10]:
%%time
dezembro['REAL'] = dezembro.apply(lambda row: corretor.ipca(row["TARIFA"], f"{int(row.MES)}/{int(row.ANO)}", "12/2022")[0], axis=1) 

CPU times: user 1min 30s, sys: 845 ms, total: 1min 30s
Wall time: 1min 34s


In [11]:
%%time
dezembro['REAL'] = dezembro.apply(lambda row: corretorOld.ipca(row["TARIFA"], f"{int(row.MES)}/{int(row.ANO)}", "12/2022")[0], axis=1) 

CPU times: user 22min 41s, sys: 15.4 s, total: 22min 56s
Wall time: 23min 44s


In [ ]:

wm = lambda x: np.average(x, weights=dezembro.loc[x.index, "ASSENTOS"])

dezembro.groupby(['ANO', 'MES']).agg(price_weighted_mean=("REAL", wm))

In [184]:
# Ver se vai dar o mesmo resultado em dezembro.
wm = lambda x: np.average(x, weights=dezembro.loc[x.index, "ASSENTOS"])

dezembro.groupby(['ANO', 'MES']).agg(price_weighted_mean=("REAL", wm))

,,price_weighted_mean
ANO,MES,
2019,12,537.315359


In [ ]:
def get_average(values, weights):
    return float(np.average(values, weights=weights))    # function created to calculate wt. median


In [116]:
df["real"] = df[1:].apply(lambda row: corretor.ipca(row["Weighted_average"], f"{int(row.MES)}/{int(row.ANO)}", "12/2022")[0], axis=1) 

In [117]:
df[df["MES"]==12]

,ANO,MES,Total,Weighted_average,real
244,2022,12,1826543,650.817925,NaN
115,2021,12,2421438,490.797236,522.98
110,2020,12,1868805,416.657935,491.66
185,2019,12,2585896,436.533702,537.32
197,2018,12,2498232,444.033223,564.44
125,2017,12,2745175,430.039663,568.77
147,2016,12,2839457,363.795988,494.65
21,2015,12,3286404,358.082682,520.90
52,2014,12,3915556,333.243424,535.55
78,2013,12,3790080,351.360833,601.69
